# SparkSQL Opeartions

## Description

SparkSQL is all about working with structured data. It extends RDD to a "DataFrame" object.

Characteristics of DataFrames:
* Contain Row objects
* Can run SQL squeries
* Has a schema (leading to more efficient storage)
* Read and write to JSON, Hive, parquet
* Communicates with JDBC/ODBC, tableau

# Datasets

* A DataFrame is really just a DataSet of row objects, i.e., `(DataSet[Row])`.

* DataSet can explicitly wrap a given struct or type, i.e., `(DataSet[Person], DataSet[(String, Double)])`
  * It knows what its columns are from the get-to.
* DataFrames schema is inferred at runtine; but a DataSet can be inferred at compile time.
  * Faster detection of errors, and better optimization.
* RDDs can be converted to DataSets with `.toDS()` method.

## Datasets are the new hotness
* The trend in Spark is to use RDDs less, and DataSets more.
* DataSets are more efficient
   * They can be serialized very efficiently - even better than Kryo.
   * Optimal execution plans can be determined at compile time.
* DataSets allow for better interoperability.
   * MLLib and Spark Streaming are moving toward using DataSets instead of RDDs for their primary API.
* DataSets simplify development
   * You can perform most SQL operations on a dataset with one line.

# Using SparkSQL in Scala

In Spark 2.0.0, you create a `SparkSession` object instead of a `SparkContext` when using SparkSQL/DataSets.
  * You can get a `SparkContext` from this session, and use it to issue SQL queries on your datasets!
  * Stop the session when you are done.
  
Example of methods with Dataframes:
  * `df.show()`: Shows the first several rows.
  * `df.select("someFieldName")`: Show a specific column.
  * `df.filter(df("someFieldName")>200)`: Slice the dataframe according to some filter criteria.
  * `df.groupBy(df("someFieldName")).mean()`: Take a groupby of some column and take the mean of it.
  * `df.rdd().map(mapperFunc)`: Map values based on some function.

In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.19:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1589311717979)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@19a96a5a


There are many ways to create a dataframe.

# Constructing DataFrame

## Method 1: RDD 

In [2]:
// Infer the schema, and register the DataSet as a table.
import spark.implicits._

import spark.implicits._


In [3]:
val rdd = {sc.parallelize(
                 Seq(
                     ("John", 19),
                     ("Smith", 23),
                     ("Sarah", 18)
                 )
              )
            }

rdd: org.apache.spark.rdd.RDD[(String, Int)] = ParallelCollectionRDD[0] at parallelize at <console>:28


In [4]:
val df = spark.createDataFrame(rdd)

df: org.apache.spark.sql.DataFrame = [_1: string, _2: int]


We see by default the `createDataFrame` method gives the column names as `_1` and `_2`. 

To define our own column names in this case, we use the `toDF()` method, which can be called on a sequence object to create a DataFrame.

In [5]:
val df_final = df.toDF("name", "age")

df_final: org.apache.spark.sql.DataFrame = [name: string, age: int]


We can print the schema.

In [6]:
df_final.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = false)



## Method 2: Constructing DataFrame from RDD Row types

In [7]:
import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.types._

import org.apache.spark.sql.{Row, SQLContext}
import org.apache.spark.sql.types._


Create a schema for the dataframe explicitly.

In [8]:
val schema = {new StructType().add(StructField("name", StringType, true))
                              .add(StructField("age", IntegerType,true))
             }

schema: org.apache.spark.sql.types.StructType = StructType(StructField(name,StringType,true), StructField(age,IntegerType,true))


In [9]:
val rdd = sc.parallelize(Seq(Row("John", 19),
                      Row("Smith", 23),
                      Row("Sarah", 18))
                     )

rdd: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = ParallelCollectionRDD[1] at parallelize at <console>:32


In [10]:
rdd.collect().foreach(println)

[John,19]
[Smith,23]
[Sarah,18]


The DataFrame is created from the RDD or Rows. 

Infer schema from the first row, create a DataFrame and print the schema

In [11]:
val df = spark.createDataFrame(rdd, schema)
df.show()

+-----+---+
| name|age|
+-----+---+
| John| 19|
|Smith| 23|
|Sarah| 18|
+-----+---+



df: org.apache.spark.sql.DataFrame = [name: string, age: int]


In [12]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)



# Loading Dataframes from files

Use the `fakefriends.csv` data set for this exercise. 

We first create the schema for the dataset.

In [24]:
val schema = {new StructType().add(StructField("id", IntegerType, true))
                              .add(StructField("name", StringType,true))
                              .add(StructField("age", IntegerType,true))
                              .add(StructField("numFriends", IntegerType,true))
              
             }

schema: org.apache.spark.sql.types.StructType = StructType(StructField(id,IntegerType,true), StructField(name,StringType,true), StructField(age,IntegerType,true), StructField(numFriends,IntegerType,true))


In [27]:
val people= spark.read
             .format("csv")
             .schema(schema)
             .load("../../data/fakefriends.csv")

people: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


In [31]:
people.show()

+---+--------+---+----------+
| id|    name|age|numFriends|
+---+--------+---+----------+
|  0|    Will| 33|       385|
|  1|Jean-Luc| 26|         2|
|  2|    Hugh| 55|       221|
|  3|  Deanna| 40|       465|
|  4|   Quark| 68|        21|
|  5|  Weyoun| 59|       318|
|  6|  Gowron| 37|       220|
|  7|    Will| 54|       307|
|  8|  Jadzia| 38|       380|
|  9|    Hugh| 27|       181|
| 10|     Odo| 53|       191|
| 11|     Ben| 57|       372|
| 12|   Keiko| 54|       253|
| 13|Jean-Luc| 56|       444|
| 14|    Hugh| 43|        49|
| 15|     Rom| 36|        49|
| 16|  Weyoun| 22|       323|
| 17|     Odo| 35|        13|
| 18|Jean-Luc| 45|       455|
| 19|  Geordi| 60|       246|
+---+--------+---+----------+
only showing top 20 rows



## Performing SQL Queries

We could use SQL queries to extract specific subsets of the data.

1. Register `people` Dataset as a temporary view in Catalog

In [32]:
people.createOrReplaceTempView("people")

2. Run the SQL query

In [33]:
val query:String = {
    """
    SELECT * 
    FROM people
    WHERE age >= 13 AND age <= 19
    """
}

val teenagers = spark.sql(query)

query: String =
"
    SELECT *
    FROM people
    WHERE age >= 13 AND age <= 19
    "
teenagers: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


In [34]:
val results = teenagers.show

+---+-------+---+----------+
| id|   name|age|numFriends|
+---+-------+---+----------+
| 21|  Miles| 19|       268|
| 52|Beverly| 19|       269|
| 54|  Brunt| 19|         5|
|106|Beverly| 18|       499|
|115|  Dukat| 18|       397|
|133|  Quark| 19|       265|
|136|   Will| 19|       335|
|225|   Elim| 19|       106|
|304|   Will| 19|       404|
|341|   Data| 18|       326|
|366|  Keiko| 19|       119|
|373|  Quark| 19|       272|
|377|Beverly| 18|       418|
|404| Kasidy| 18|        24|
|409|    Nog| 19|       267|
|439|   Data| 18|       417|
|444|  Keiko| 18|       472|
|492|  Dukat| 19|        36|
|494| Kasidy| 18|       194|
+---+-------+---+----------+



results: Unit = ()


# Datasets

To work with datasets from a csv file, note that Spark imports csv as a DataFrame. To demonstrate a more complicated example, let us define a custom data type `Person` that consists of the 4 columns of the csv as mentioned in the `schema` in the above example.

In [36]:
case class Person(id: Int, name:String, age:Int, numFriends:Int)

defined class Person


For Spark 2.0 or above, to go from dataFrame to dataset requires an encoder to cast this user-defined type to the dataFrame.

In [46]:
import org.apache.spark.sql.Encoders

import org.apache.spark.sql.Dataset
import org.apache.spark.sql.Encoders


In [47]:
val encoder = Encoders.product[Person]

encoder: org.apache.spark.sql.Encoder[Person] = class[id[0]: int, name[0]: string, age[0]: int, numFriends[0]: int]


We import the csv file as usually but without a schema like the above example.

In [37]:
val people= spark.read
             .format("csv")
             .load("../../data/fakefriends.csv")

people: org.apache.spark.sql.DataFrame = [_c0: string, _c1: string ... 2 more fields]


We now convert the dataFrame into a Dataset of type `Person`.

In [48]:
val peopleDS = df.as(encoder)

peopleDS: org.apache.spark.sql.Dataset[Person] = [id: int, name: string ... 2 more fields]


Here are the following things we can do with dataset.

Here is our schema:

In [49]:
peopleDS.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- numFriends: integer (nullable = true)



Let us select the name column:

In [50]:
peopleDS.select("name").show()

+--------+
|    name|
+--------+
|    Will|
|Jean-Luc|
|    Hugh|
|  Deanna|
|   Quark|
|  Weyoun|
|  Gowron|
|    Will|
|  Jadzia|
|    Hugh|
|     Odo|
|     Ben|
|   Keiko|
|Jean-Luc|
|    Hugh|
|     Rom|
|  Weyoun|
|     Odo|
|Jean-Luc|
|  Geordi|
+--------+
only showing top 20 rows



Filter out anyone over 21:

In [53]:
peopleDS.filter(peopleDS("age") < 21).show()

+---+-------+---+----------+
| id|   name|age|numFriends|
+---+-------+---+----------+
| 21|  Miles| 19|       268|
| 48|    Nog| 20|         1|
| 52|Beverly| 19|       269|
| 54|  Brunt| 19|         5|
| 60| Geordi| 20|       100|
| 73|  Brunt| 20|       384|
|106|Beverly| 18|       499|
|115|  Dukat| 18|       397|
|133|  Quark| 19|       265|
|136|   Will| 19|       335|
|225|   Elim| 19|       106|
|304|   Will| 19|       404|
|327| Julian| 20|        63|
|341|   Data| 18|       326|
|349| Kasidy| 20|       277|
|366|  Keiko| 19|       119|
|373|  Quark| 19|       272|
|377|Beverly| 18|       418|
|404| Kasidy| 18|        24|
|409|    Nog| 19|       267|
+---+-------+---+----------+
only showing top 20 rows



Group by age:

In [54]:
peopleDS.groupBy("age").count().show()

+---+-----+
|age|count|
+---+-----+
| 31|    8|
| 65|    5|
| 53|    7|
| 34|    6|
| 28|   10|
| 26|   17|
| 27|    8|
| 44|   12|
| 22|    7|
| 47|    9|
| 52|   11|
| 40|   17|
| 20|    5|
| 57|   12|
| 54|   13|
| 48|   10|
| 19|   11|
| 64|   12|
| 41|    9|
| 43|    7|
+---+-----+
only showing top 20 rows



Make everyone 10 years older:

In [56]:
peopleDS.select(peopleDS("name"), peopleDS("age") + 10).show()

+--------+----------+
|    name|(age + 10)|
+--------+----------+
|    Will|        43|
|Jean-Luc|        36|
|    Hugh|        65|
|  Deanna|        50|
|   Quark|        78|
|  Weyoun|        69|
|  Gowron|        47|
|    Will|        64|
|  Jadzia|        48|
|    Hugh|        37|
|     Odo|        63|
|     Ben|        67|
|   Keiko|        64|
|Jean-Luc|        66|
|    Hugh|        53|
|     Rom|        46|
|  Weyoun|        32|
|     Odo|        45|
|Jean-Luc|        55|
|  Geordi|        70|
+--------+----------+
only showing top 20 rows

